In [2]:
# 1
# Chance Wiese

In [3]:
# 2
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    database='corgirace',
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

cursor = conn.cursor()

query = '''
SELECT *
FROM corgi;
'''

df = pd.read_sql(query, conn, index_col='corgid')
print(df)

                         corgname  breed gender  weight  age fromfile
corgid                                                               
1                   Snappy Ginger    Pem      F      21  3.5     None
2                   Golden Graham    Pem      M      30  2.0     None
3                          Kraken    Pem     SF      22  3.5     None
4                     Snarla June    Pem     SF      29  3.5     None
5                 Smiley Shortdog  Cardi     NM      33  4.5     None
6                         Stanley    Pem      M      33  3.5     None
7                Holden Corgfield    Pem      M      25  4.0     None
8           Penny the Shedmonster    Pem      F      20  5.0     None
9       Baroness von Fluffyshanks    Pem     SF      22  4.0     None
10                     Prometheus    Pem      M      27  5.0     None
11                 Foxy Stumptail    Pem     NM      29  5.5     None
12                      Drederick    Pem      M      29  4.5     None
13              Lady

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10032\1924361383.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, index_col='corgid')


In [4]:
# 3
print("corgname: Can be a string of up to 30 characters.")
print("breed: Can only be 1 of 2 options: 'Pem' or 'Cardi.'")
print("gender: Can only be 1 of 4 options: SF, NM, M, F.")
print("weight: Positive integer less than 50, as corgis don't get that big.")
print("age: Positive number with 1 decimal place, allowing for half-year precision (e.g., 3.0 or 3.5 years).")

corgname: Can be a string of up to 30 characters.
breed: Can only be 1 of 2 options: 'Pem' or 'Cardi.'
gender: Can only be 1 of 4 options: SF, NM, M, F.
weight: Positive integer less than 50, as corgis don't get that big.
age: Positive number with 1 decimal place, allowing for half-year precision (e.g., 3.0 or 3.5 years).


In [5]:
# 4
cursor.execute('''
CREATE TABLE IF NOT EXISTS ingest (
    filename VARCHAR(255) PRIMARY KEY,
    whendone TIMESTAMP NOT NULL
);
''')

conn.commit()

In [6]:
# 5
cursor.execute('''
ALTER TABLE corgi
ADD COLUMN IF NOT EXISTS fromfile VARCHAR(255);
''')

conn.commit()

query = '''
SELECT corgname, fromfile
FROM corgi;
'''

df = pd.read_sql(query, conn)
print(df)

                     corgname fromfile
0               Snappy Ginger     None
1               Golden Graham     None
2                      Kraken     None
3                 Snarla June     None
4             Smiley Shortdog     None
5                     Stanley     None
6            Holden Corgfield     None
7       Penny the Shedmonster     None
8   Baroness von Fluffyshanks     None
9                  Prometheus     None
10             Foxy Stumptail     None
11                  Drederick     None
12          Lady Wigglebottom     None
13                   Jedediah     None
14               Miss Juniper     None
15        Mary Queen of Corgs     None
16                    Oatmeal     None
17                 Cowboy Dan     None
18                Gareth Bale     None
19              Winston Smith     None
20                      Bucco     None
21                   Fruitcup     None
22                  Sweet Dee     None
23                  Stonewall     None
24             Hywel the 

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10032\2746114488.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [7]:
# 6
csvname = "newcorgis-20220618.csv"

query = f'''
SELECT COUNT(*)
FROM ingest
WHERE filename = '{csvname}'
'''

cursor.execute(query)
count = cursor.fetchone()[0]

if count > 0:
    print(f"The file '{csvname}' has already been processed.")
else:
    print(f"The file '{csvname}' has been accepted for processing.")

    df2 = pd.read_csv(csvname)

    # Data Cleaning
    df2['Breed'] = df2['Breed'].replace({'PWC': 'Pem', 'CWC': 'Cardi'}) # Breed
    df2['Gender'] = df2['Gender'].replace({'Fem': 'F'}) # Gender
    df2['Weight'] = df2['Weight'].round().astype(int) # Weight
    df2['Age'] = (df2['Age'] * 2).round() / 2 # Age

    print("\nCleaned data:\n", df2, "\n")

    # Load corgi
    for x in df2.index:
        cursor.execute(f'''
        INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
        VALUES (
            '{df2['Name'].loc[x]}',
            '{df2['Breed'].loc[x]}',
            '{df2['Gender'].loc[x]}',
            '{df2['Weight'].loc[x]}',
            '{df2['Age'].loc[x]}',
            '{csvname}');
        ''')
        print(f"Record created for {df2['Name'].loc[x]}")

    conn.commit()

    # Load ingest
    cursor.execute(f'''
    INSERT INTO ingest (filename, whendone)
    VALUES ('{csvname}', current_timestamp);
    ''')

    conn.commit()
    print(f"Ingestion complete for {csvname}.")

    query = f'''
        SELECT *
        FROM ingest
        WHERE filename = '{csvname}';
        '''
    df_ingest = pd.read_sql(query, conn)
    print("\nIngest Validation:\n", df_ingest)

The file 'newcorgis-20220618.csv' has been accepted for processing.

Cleaned data:
               Name  Breed Gender  Weight  Age
0         Francine    Pem      F      24  3.5
1         Prudence    Pem     SF      29  4.0
2        Fort Knox  Cardi     NM      34  3.5
3          Gruyere    Pem     SF      22  1.5
4     Sir Barxalot    Pem      M      26  2.0
5      Tuning Fork  Cardi     NM      29  2.0
6          Wiffles    Pem     SF      23  5.0
7           Wumpus    Pem      F      26  1.0
8  Qween Kanonball    Pem      F      29  3.5
Record created for Francine
Record created for Prudence
Record created for Fort Knox
Record created for Gruyere
Record created for Sir Barxalot
Record created for Tuning Fork
Record created for Wiffles
Record created for Wumpus
Record created for Qween Kanonball
Ingestion complete for newcorgis-20220618.csv.


C:\Users\Chanc\AppData\Local\Temp\ipykernel_10032\3875003940.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ingest = pd.read_sql(query, conn)



Ingest Validation:
                  filename                   whendone
0  newcorgis-20220618.csv 2024-10-07 03:15:29.154882


In [15]:
# 7
query = f'''
SELECT *
FROM corgi
WHERE fromfile = '{csvname}';
'''
dfv = pd.read_sql(query, conn, index_col='corgid')
print("\nCorgi Validation:\n", dfv)

C:\Users\Chanc\AppData\Local\Temp\ipykernel_10032\1607585902.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfv = pd.read_sql(query, conn, index_col='corgid')


InterfaceError: connection already closed

In [13]:
cursor.close()
conn.close()